In [2]:
import numpy as np

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:

import pickle
def load_pickle(file):
     with open(file, mode='rb') as f:
        try:
            obj = pickle.load(f)
            return obj
        except Exception as e:
            print(e)
mnist_X = load_pickle('mnist_X.pickle')
mnist_y = load_pickle('mnist_y.pickle')

In [4]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def deriv_sigmoid(x):
    return sigmoid(x)*(1 - sigmoid(x))

In [28]:
def ReLU(x):
    return np.where(x>0,x,0)

def deriv_ReLU(x):
    return np.where(x>0,1,0)

In [5]:
def softmax(x):
    ex = np.exp(x)
    return ex/np.sum(ex,axis=1)[:,np.newaxis]

In [6]:
train_X, test_X, train_y, test_y = train_test_split(mnist_X[:10000], mnist_y[:10000],
                                                    test_size=0.2,
                                                    random_state=43)

In [7]:
class Layer:
    def __init__(self, in_dim, out_dim, function, deriv_function):
        self.W = np.random.uniform(low= -0.08, high= 0.08, size=(in_dim, out_dim)).astype("float32")
        self.b = np.zeros(out_dim).astype("float32")
        self.function = function
        self.deriv_function = deriv_function
        self.u = None
        self.delta = None
        
    def f_prop(self, x):
        self.u = np.dot(x, self.W) + self.b
        self.z = self.function(self.u)
        return self.z
    
    def b_prop(self, delta, W):
        self.delta = self.deriv_function(self.u)*np.dot(delta, W.T)
        return self.delta
    
def f_props(layers, x):
    z = x
    for layer in layers:
        z = layer.f_prop(z)
    return z

def b_props(layers, delta):
    for i, layer in enumerate(layers[::-1]):
        if i == 0:
            layer.delta = delta
        else:
            delta = layer.b_prop(delta, _W)
        _W = layer.W

In [8]:
def train(X, t, eps=1.,l=1e-5):
    y = f_props(layers, X)
    delta = y - t
    b_props(layers, delta)

    z = X
    for i, layer in enumerate(layers):
        dW = np.dot(z.T, layer.delta)+l*layer.W
        db = np.dot(np.ones(z.shape[0]), layer.delta)+l*layer.b
        layer.W = layer.W - eps*dW
        layer.b = layer.b - eps*db
        z = layer.z

def test(X, t):
    y = f_props(layers, X)
    return y

In [10]:
mean = train_X.mean(axis=0)
std = train_X.std(axis=0)
train_X_std = train_X
test_X_std = test_X

In [36]:
layers = [Layer(784, 300, ReLU, deriv_ReLU),
          Layer(300, 10, softmax, None)]
batch = 101
for epoch in range(100):
    i = 0
    while i+batch < len(train_X_std):
        train(train_X_std[i:i+batch], train_y[i:i+batch],1e-2)
        i+=batch
    train(train_X_std[i:], train_y[i:],1e-1)
    pred_train_y = test(train_X_std, train_y)
    print(accuracy_score(np.argmax(pred_train_y,axis=1), np.argmax(train_y,axis=1)))
    pred_y = test(test_X_std, test_y)
    print(accuracy_score(np.argmax(pred_y,axis=1), np.argmax(test_y,axis=1)))

0.44225
0.425
0.86775
0.8335
0.965625
0.9375
0.971625
0.94
0.980375
0.9455
0.985625
0.947
0.98925
0.946
0.992
0.948
0.99475
0.9505
0.996
0.9515
0.996875
0.951
0.9985
0.9525
0.9995
0.953
1.0
0.953
1.0
0.954
1.0
0.9545
1.0
0.9545
1.0
0.954
1.0
0.954
1.0
0.954
1.0
0.954
1.0
0.9545
1.0
0.955
1.0
0.955
1.0
0.9545
1.0
0.954
1.0
0.954
1.0
0.954
1.0
0.954
1.0
0.954
1.0
0.954
1.0
0.954
1.0
0.9535
1.0
0.9535
1.0
0.9535
1.0
0.9535
1.0
0.9535
1.0
0.9535
1.0
0.9535
1.0
0.9535
1.0
0.9535
1.0
0.9535
1.0
0.9535
1.0
0.954
1.0
0.9535
1.0
0.954
1.0
0.954
1.0
0.954
1.0
0.9545
1.0
0.9545
1.0
0.9545
1.0
0.9545
1.0
0.9545
1.0
0.9545
1.0
0.9545
1.0
0.955
1.0
0.9545
1.0
0.955
1.0
0.9555
1.0
0.955
1.0
0.9555
1.0
0.956
1.0
0.9555
1.0
0.956
1.0
0.956
1.0
0.9565
1.0
0.956
1.0
0.9565
1.0
0.9565
1.0
0.956
1.0
0.956
1.0
0.956
1.0
0.956
1.0
0.956
1.0
0.956
1.0
0.956
1.0
0.956
1.0
0.956
1.0
0.956
1.0
0.956
1.0
0.9565
1.0
0.956
1.0
0.956
1.0
0.956
1.0
0.956
1.0
0.956
1.0
0.9565
1.0
0.9565
1.0
0.9565
1.0
0.9565
1.0
0.956

In [ ]:
layers[0].b